In [23]:
class B():
    def __init__(self,b1):
        self.b1=b1
        print(b1)

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/opt/conda/envs/venv/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/venv/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/venv/lib/python3.9/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/opt/conda/envs/venv/lib/python3.9/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/opt/conda/envs/venv/lib/python3.9/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


In [25]:
dictionary={'key1':2}
class A():
    def __init__(self,key1):
        self.key1=key1
        print(key1)
        binstance=B(b1=key1)
        
temp=A(**dictionary)

2
2


In [26]:
import torch
import numpy as np
import segmentation_models_pytorch as smp

import torch.nn as nn
from torchmetrics.functional import accuracy

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

from torchlightning_module import torch_lightning_DataModule
from dataset_module import DatasetModule

torch.set_float32_matmul_precision('medium')

###
class SemanticSegmentationModel(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        for key in hparams.keys():
             self.hparams[key]=hparams[key]

        self.loss_fn = nn.CrossEntropyLoss()
        self.save_hyperparameters()
        
        self.model = smp.Unet( #modifiy in-channels and output. 
           encoder_name=self.hparams['encoder_name'], 
           classes=self.hparams['classes'], 
           activation=self.hparams['activation'], 
           encoder_weights=self.hparams['encoder_weights'],
            in_channels=self.hparams['in_channels'],
              )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self.model(images)
        labels = labels.squeeze(1)
        loss = self.loss_fn(outputs, labels)
        
        
        #outputs : output of model > logits /// labels > y ///images > x 
        self.log('train_loss',loss, on_step=False, on_epoch=True)
        return loss
    
    
#     def training_epoch_end(self, outputs):
#         avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
#         self.log("avg_train_loss", avg_loss)
#         wandb.log({"avg_train_loss": avg_loss})


    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self.model(images)
        labels = labels.squeeze(1)
        #print(f'{labels.shape} is shape of label in the val step method')
        #print(f'{outputs.shape} is shape of outputs in the val step method')

        loss = self.loss_fn(outputs, labels)
        # Log loss and accuracy
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        
        return {'val_loss': loss}

#     def validation_epoch_end(self, outputs):
#         avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#         logs = {'val_loss': avg_loss}
#         return {'val_loss': avg_loss, 'log': logs}

        # def validation_epoch_end(self, outputs):
        #     avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        #     logs = {'val_loss': avg_loss}
        #     self.log('val_loss', avg_loss)



    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

    
# def lr_schedule(step):
#    lr = 0.001
#    if step < 10:
#        return lr
#    elif step < 20:
#        return lr / 2
#    else:
#        return lr / 4

#lr_scheduler = pl.callbacks.LearningRateScheduler(lr_schedule)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=500,
    mode='min'
)


# Define your hyperparameters
hparams = {
    'encoder_name': 'resnet34', #resnet34,50
    'classes': 13,
    'activation': 'softmax',
    'encoder_weights': 'imagenet', #imagenet
    'lr': 0.001,#  
    'in_channels':5,

}

# Initialize your model
model = SemanticSegmentationModel(hparams)
data=torch_lightning_DataModule(batch_size=32,num_workers=24,pin_memory=True)

wandb_logger = WandbLogger(project='phase2_semantic_segmentation_initial_run')

trainer = pl.Trainer(max_epochs=5,
                     accelerator='gpu',
                     callbacks=[early_stopping],
                     logger=wandb_logger
                    ) #lr_scheduler

trainer.fit(model,data)

/opt/conda/envs/venv/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/venv/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory ./phase2_semantic_segmentation_initial_run/cqvzf55a/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | loss_fn | CrossEntropyLoss | 0     
1 | model   | Unet             | 24.4 M
----------------------------

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/envs/venv/lib/python3.9/site-packages/segmentation_models_pytorch/base/modules.py:116: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


Training: 0it [00:00, ?it/s]

/opt/conda/envs/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [7]:
import yaml
from pytorch_lightning.callbacks import EarlyStopping #, ModelCheckpoint


with open("hyperparameters.yaml", "r") as stream:
    try:
        hyperparameters = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        



In [10]:
**hyperparameters["model"]

SyntaxError: invalid syntax (2110965215.py, line 1)